### Численное решение нелинейных уравнений и систем

Нелинейные уравнения и системы уравнений решаются с применением
итерационных методов.
$$\{x^0, x^1, x^2, ...\}$$

Поиск локализованного корня уравнения
$$ f(x) = 0$$
Системы
$$
\begin{cases}
f_1(x_1, x_2,\dots,x_n)=0 \\
f_2(x_1, x_2,\dots,x_n)=0 \\
\dots \\
f_n(x_1, x_2,\dots,x_n) = 0
\end{cases} \quad \Leftrightarrow \quad f(x) = 0
$$

#### Метод деления отрезка пополам (Бисекции) для скалярного уравнения

Если $f$ непрерывна на отрезке $[a,\;b]$ и $f(a)f(b) < 0$, то существует $p \in [a,\;b]$ являющаяся корнем уравнения $f(x) = 0$.

На каждой итерации берется
$$p = a + \frac{b - a}{2}$$
Если $f(a)f(p) < 0$, то переходим к отрезку $[a_1,\;b_1] = [a,\;p]$, в противном случае к отрезку $[a_1,\;b_1] = [p,\;b]$ и т.д.

In [45]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import matplotlib
matplotlib.rc("animation", html="jshtml")

a = [0.2]
b = [1.3]
p = []
f = lambda x: np.cos(x) - x ** 3 if x is not None else None

FA = f(a[-1])
for i in range(20):
    p.append(None)
    a.append(a[-1])
    b.append(b[-1])
    p.append(a[-1] + (b[-1] - a[-1]) / 2)
    FP = f(p[-1])
    if np.isclose(FP, 0.0, atol=0.001) or (b[-1] - a[-1]) / 2 < 0.1:
        break
    if np.sign(FA) * np.sign(FP) > 0:
        a.append(p[-1])
        b.append(b[-1])
        FA = FP
    else:
        a.append(a[-1])
        b.append(p[-1])

count = 0
subcount = 0
a_label = []
b_label = []
p_label = []
for _ in range(len(a)):
    a_label.append(r"$a_{}$".format(count))
    b_label.append(r"$b_{}$".format(count))
    p_label.append(r"$p_{}$".format(count))
    subcount += 1
    if subcount == 2:
        subcount = 0
        count += 1

fig, ax = plt.subplots(figsize=(7, 5))
# set the x-spine (see below for more info on `set_position`)
ax.spines["left"].set_position("zero")
# turn off the right spine/ticks
ax.spines["right"].set_color("none")
ax.yaxis.tick_left()
# set the y-spine
ax.spines["bottom"].set_position("zero")
# turn off the top spine/ticks
ax.spines["top"].set_color("none")
ax.xaxis.tick_bottom()

ax.set_xlim(-0.2, 2.0)
ax.set_ylim(-4, 4)

x = np.linspace(-1, 3, 100)

lines = []
lines.append(ax.plot(x, f(x), "k-", label=r"$\cos(x) - x^3$")[0])
lines.append(ax.plot([], [], "k--")[0])
lines.append(ax.plot([], [], "k--")[0])
lines.append(ax.plot([], [], "r--")[0])
lines.append(ax.text([], [], "", fontsize=20))
lines.append(ax.text([], [], "", fontsize=20))
lines.append(ax.text([], [], "", fontsize=20, color="r"))

ax.legend(loc='upper right');

true_count = 0


def animate(i):
    lines[1].set_data([a[i], a[i]], [0.0, f(a[i])])
    lines[2].set_data([b[i], b[i]], [0.0, f(b[i])])
    lines[3].set_data([p[i], p[i]], [0.0, f(p[i])])

    if f(a[i]) > 0:
        lines[4].set_y(f(a[i]) + 0.2)
    else:
        lines[4].set_y(0.2)

    if f(b[i]) > 0:
        lines[5].set_y(f(b[i]) + 0.2)
    else:
        lines[5].set_y(0.2)

    if p[i] is not None and f(p[i]) > 0:
        lines[6].set_y(f(p[i]) + 0.2)
    else:
        lines[6].set_y(0.2)

    if p[i] is not None:
        if np.isclose(a[i], p[i]):
            lines[4].set_text("")
        elif np.isclose(a[i], b[i]):
            lines[5].set_text("")
        lines[6].set_x(p[i] - 0.05)
        lines[6].set_text(p_label[i])
    else:
        lines[4].set_x(a[i] - 0.05)
        lines[5].set_x(b[i] - 0.05)
        lines[4].set_text(a_label[i])
        lines[5].set_text(b_label[i])
        lines[6].set_text("")

    return

plt.close("all")
ani1 = FuncAnimation(fig, animate, frames=len(a) - 2, interval=1600)
ani1

Возможные критерии остановки
$$b - a < \epsilon$$
$$\vert p_n - p_{n-1} \vert < \epsilon$$
$$\dfrac{\vert p_n - p_{n-1} \vert}{\vert p_n \vert} < \epsilon$$
$$\vert f\left(p_n\right)\vert < \epsilon$$

Оценка сходимости
$$\vert p_n - p \vert < \frac{1}{2}\left(b_n - a_n\right) = \frac{b-a}{2^n}$$
$$p_n = p + O\left(\frac{1}{2^n}\right)$$

#### Метод простой итерации

$$f(x) = 0 \iff x = F(x)$$
$$x_{n+1} = F(x_n), \quad n = 0, 1, \ldots$$

**Определение**

Отображение $y = F(x)$ называется сжимающим, если
$$ \rho(F(x),\;F(y)) \leq q\rho(x,\;y), \quad 0<q<1 \quad \forall x,\;y$$
$\rho(\cdot\;,\cdot)$ - расстояние. В нормированном пространстве $\rho(x,\;y)=\|x-y\|.$

**Теорема** О сжимающем отображении

Последовательность $\{x_n\},\; n=0,1,\dots$ элементов $n$-мерного метрического пространства, порожденная итерационным процессом
$$ x_{n+1} = F(x_n), \quad n = 0, 1, \ldots, \quad x_0 = a$$
сходится к единственному решению $x^*$ уравнения $x=F(x)$, если отображение 
$$y=F(x)$$ 
является сжимающим. При этом справедлива оценка
$$
\rho(x_n, x^*) \le \frac{q^n}{1 - q} \rho(x_1, x_0).
$$


> Решение $x^*$ уравнения $x = F(x)$ называется *неподвижной точкой отображения*

**Теорема** Достаточное условие сходимости МПИ

Пусть отображение $F : \mathbb{R}^n \to \mathbb{R}^n$ имеет единственную неподвижную точку $x^* = F(x^*)$ и непрерывно дифференцируемо в некоторой её окрестности. 
$$
J=F'(x) = \left[\begin{array}{ccc}
\frac{\partial F_1(x)}{\partial x_1} & \ldots & \frac{\partial F_1(x)}{\partial x_n} \\
\ldots & \ldots & \ldots \\
\frac{\partial F_n(x)}{\partial x_1} & \ldots & \frac{\partial F_n(x)}{\partial x_n}
\end{array}\right]
$$
Тогда,  если $\|J\| < 1$, то отображение $y=F(x)$ является сжимающим в некоторой окрестности точки $x^*$ и метод простой итерации сходится к $x^*$ для $\forall x_0$ из этой окрестности.

##### Метод простой итерации с параметром (Релаксации)

$$ F(x) = x + \tau f(x) $$
$$ x_{n+1} = x_n + \tau f(x_n), \quad n=0,1,\dots $$

In [54]:
p = [0.4]

f = lambda x: np.cos(x) - x ** 3 if x is not None else None
fright = lambda x: x + 0.1 * f(x) if x is not None else None

for i in range(15):
    x = p[-1] if p[-1] is not None else p[-2]
    xold = x
    x = fright(x)
    p.append(None)
    p.append(x)
    if np.abs(x - xold) < 1e-4:
        break

count = 0
subcount = 0
p_label = []
for _ in range(len(p)):

    p_label.append(r"$p_{{{}}}$".format(count))

    subcount += 1
    if subcount == 2:
        subcount = 0
        count += 1

fig, ax = plt.subplots(figsize=(8, 6))
ax.spines["left"].set_position("zero")
ax.spines["right"].set_color("none")
ax.yaxis.tick_left()
ax.spines["bottom"].set_position("zero")
ax.spines["top"].set_color("none")
ax.xaxis.tick_bottom()

ax.set_xlim(0, 1.25)
ax.set_ylim(0, 1.25)

x = np.linspace(0.0, 1.25, 100)

lines = []
lines.append(ax.plot(x, fright(x), "k-", label=r"$F(x)$")[0])
lines.append(ax.plot([], [], "k--")[0])
lines.append(ax.plot([], [], "r-")[0])
lines.append(ax.plot([], [], "r-")[0])
lines.append(ax.plot(x, x, "k--")[0])
lines.append(ax.text([], [], "", fontsize=20))
lines.append(ax.plot(x, f(x), "g-", label=r"$\cos(x) - x^3$")[0])

ax.legend(loc='upper left');

def animate(i):

    if p[i] is None:
        lines[2].set_data([p[i - 1], p[i + 1]], [fright(p[i - 1]), p[i + 1]])
        lines[3].set_data([p[i + 1], p[i + 1]], [p[i + 1], fright(p[i + 1])])

    if p[i] is not None:
        lines[1].set_data([p[i], p[i]], [0.0, fright(p[i])])
        if fright(p[i]) > 0:
            lines[5].set_y(fright(p[i]) + 0.05)
            lines[5].set_x(p[i] - 0.05)
            lines[5].set_text(p_label[i])
        else:
            lines[5].set_y(fright(p[i]) - 0.05)
            lines[5].set_x(p[i] - 0.05)
            lines[5].set_text(p_label[i])

    return


plt.close("all")
ani2 = FuncAnimation(fig, animate, frames=len(p), interval=1600)
ani2

#### Метод ньютона
Пусть нам известно приближенное решение $x_n$ уравнения $f(x) = 0$.
$$ f(x) \approx f(x_n) + f'(x_n) (x - x_n) $$
$$ f(x_n) + f'(x_n) (x - x_n) = 0 $$
В случае, когда <ins>задана функция и известна её производная</ins>
$$x_{n+1} = x_n - \frac{f(x_n)}{f'\left(x_n\right)}$$
Для системы 
$$x_{n+1} = x_n - J^{-1}f(x_n)$$
$$
J = \left[\begin{array}{ccc}
\frac{\partial F_1(x)}{\partial x_1} & \ldots & \frac{\partial F_1(x)}{\partial x_n} \\
\ldots & \ldots & \ldots \\
\frac{\partial F_n(x)}{\partial x_1} & \ldots & \frac{\partial F_n(x)}{\partial x_n}
\end{array}\right]
$$
Критерий остановки может быть таким
$$\| x_{n+1} - x_n \| < \epsilon$$

In [56]:
p = [0.4]

f = lambda x: np.cos(x) - x ** 3 if x is not None else None
fprime = lambda x: -3 * x ** 2 - np.sin(x) if x is not None else None

for i in range(15):

    x = p[-1] if p[-1] is not None else p[-2]

    xold = x
    x = x - f(x) / fprime(x)
    p.append(None)
    p.append(x)

    if np.abs(x - xold) < 1e-4:
        break

count = 0
subcount = 0
p_label = []
for _ in range(len(p)):

    p_label.append(r"$p_{}$".format(count))

    subcount += 1
    if subcount == 2:
        subcount = 0
        count += 1

fig, ax = plt.subplots(figsize=(8, 6))
ax.spines["left"].set_position("zero")
ax.spines["right"].set_color("none")
ax.yaxis.tick_left()
ax.spines["bottom"].set_position("zero")
ax.spines["top"].set_color("none")
ax.xaxis.tick_bottom()

ax.set_xlim(-0.2, 2.0)
ax.set_ylim(-4, 4)

x = np.linspace(-1, 3, 100)

lines = []
lines.append(ax.plot(x, f(x), "k-", label=r"$\cos(x) - x^3$")[0])
lines.append(ax.plot([], [], "k--")[0])
lines.append(ax.plot([], [], "r-")[0])
lines.append(ax.text([], [], "", fontsize=20))


def animate(i):

    lines[1].set_data([p[i], p[i]], [0.0, f(p[i])])
    if p[i] is None:
        lines[2].set_data(x, fprime(p[i - 1]) * (x - p[i - 1]) + f(p[i - 1]))
        lines[3].set_text("")

    if p[i] is not None:
        if f(p[i]) > 0:
            lines[3].set_y(f(p[i]) + 0.2)
            lines[3].set_x(p[i] - 0.05)
            lines[3].set_text(p_label[i])
        else:
            lines[3].set_y(f(p[i]) - 0.4)
            lines[3].set_x(p[i] - 0.05)
            lines[3].set_text(p_label[i])

    return


plt.close("all")
ani2 = FuncAnimation(fig, animate, frames=len(p), interval=1600)
ani2

Оценка сходимости

Пусть последовательность приближений $\{x_{k}\}$ сходится к некоторому значению $x^{\star}$.

$$x^\star = x_k + \epsilon_k$$
Распишем в ряд Тейлора
\begin{align*}
0 = f(x^{\star}) &= f(x_k) + f^\prime(x_k)\epsilon_k + \frac{1}{2} f^{\prime\prime}(\xi_k)
\end{align*}
Из метода Ньютона
$$0 = f(x_k) + f'(x_k)(x_{k+1}-x_k)$$
Возьмём разницу
$$f^\prime(x_k)(x^{\star} - x_{k+1}) + \frac{1}{2} f^{\prime\prime}(\xi_k)$$
Получаем
\begin{align*}
\epsilon_{k+1} &= -\frac{f''(\xi_k)}{2f'(x_k)}\epsilon_k^2
\end{align*}

Получили квадратичную сходимость метода Ньютона при наличии второй производной и отличной от нуля первой производной, что быстрее, чем для методов, рассмотренных ранее.

**Теорема** Сходимость в $\R$

Пусть $x^*$ - корень уравнения $f(x) = 0$ и 
- $f \in C^2[x^* - \delta, x^* + \delta]$
- $f'(x) \ne 0$ при $x \in [x^* - \delta, x^* + \delta]$
- $\gamma \equiv  \frac{\displaystyle  \max_{|x-x^*|\le\delta} \big|f''(x)\big|}{\displaystyle 2 \min_{|x-x^*|\le \delta} \big|f'(x)\big|} \ne 0$
Тогда $\forall \epsilon:\; 0 < \epsilon < \min\{\delta, \gamma^{-1}\}$ метод Ньютона сходится при любом начальном приближении $x_0 \in [x^*-\epsilon, x^*+\epsilon]$ и верно:
- $\lvert e_{k+1}\rvert \le \gamma \lvert e_k \rvert^2 $
- $\lvert e_k \rvert \le \gamma^{-1} (\gamma \lvert e_0 \rvert)^{2^k} $

**Теорема** Сходимость в $\R^n$
$$x_{k+1} = x_k - \left[f'(x_k)\right]^{-1} f(x_k)$$
$f : \mathbb{R}^n \to \mathbb{R}^n$, $f(x^*) = 0$ и в $B_\delta(x^*) = \{x: \Vert x - x^* \Vert \le \delta\}$ $f$ - непрерывно дифференцируемо, якобиан $f$ существует и невырожден, и $\forall x,y \in B_\delta$:
$$
\Vert f'(x) - f'(y) \Vert \le c \Vert x - y \Vert, \; c >0
$$
Пусть $\displaystyle \gamma = c \max_{\Vert x^* - x\Vert \le \delta} \Vert [f'(x)^{-1}] \Vert$, $0 < \epsilon < \min\{\delta, \gamma^{-1}\}$.
Тогда $\forall x_0 \in B_\epsilon(x^*)$ метод Ньютона сходится и 
$$
\begin{align*}
    \Vert e_{k+1} \Vert \le \gamma \Vert e_k \Vert^2, \quad \Vert e_k \Vert \le \gamma^{-1} (\gamma \Vert e_0 \Vert)^{2^k}
\end{align*}
$$

#### Метод секущих (Хорд)

Попробуем в методе Ньютона заменить производную на её приближенное значение
$$
f'(x_{n-1}) = \lim_{x\rightarrow x_{n-1}}\frac{f(x_{n-1})-f(x_{n-2})}{x_{n-2}-x_{n-1}} \approx \frac{f(x_{n-1})-f(x_{n-2})}{x_{n-2}-x_{n-1}}
$$

In [59]:
def secant_zero_animate(p0, p1):

    f = lambda x: np.cos(x) - x ** 3 if x is not None else None

    p0 = [p0]
    p1 = [p1]
    pnew = [None]

    q0 = f(p0[-1])
    q1 = f(p1[-1])

    for _ in range(100):
        if p1 is not None and p0 is not None:
            p = p1[-1] - q1 * (p1[-1] - p0[-1]) / (q1 - q0)
        if np.abs(p - p1[-1]) < 0.05:
            break
        p0.append(None)
        p1.append(None)

        if p0[-1] is not None:
            p0.append(p1[-1])
        else:
            p0.append(p1[-2])
        p1.append(p)

        q0 = q1
        q1 = f(p)

        pnew.append(p)
        pnew.append(None)

    count = 0
    subcount = 0
    p0_label = []
    p1_label = []
    pnew_label = []
    for _ in range(len(p0)):

        p0_label.append(r"$p_{}$".format(count))
        p1_label.append(r"$p_{}$".format(count + 1))
        pnew_label.append(r"$p_{}$".format(count + 2))

        subcount += 1
        if subcount == 2:
            subcount = 0
            count += 1

    fig, ax = plt.subplots(figsize=(8, 6))
    ax.spines["left"].set_position("zero")
    ax.spines["right"].set_color("none")
    ax.yaxis.tick_left()
    ax.spines["bottom"].set_position("zero")
    ax.spines["top"].set_color("none")
    ax.xaxis.tick_bottom()

    ax.set_xlim(-0.2, 2.0)
    ax.set_ylim(-4, 4)

    x = np.linspace(-1, 3, 100)

    lines = []
    lines.append(ax.plot(x, f(x), "k-", label=r"$\cos(x) - x^3$")[0])
    lines.append(ax.plot([], [], "r--")[0])
    lines.append(ax.plot([], [], "k--")[0])
    lines.append(ax.plot([], [], "k--")[0])
    lines.append(ax.plot([], [], "r-")[0])
    lines.append(ax.text([], [], "", fontsize=20))
    lines.append(ax.text([], [], "", fontsize=20))
    lines.append(ax.text([], [], "", fontsize=20, color="r"))

    def animate(i):

        if p0[i] is not None:
            lines[2].set_data([p0[i], p0[i]], [0.0, f(p0[i])])
            lines[3].set_data([p1[i], p1[i]], [0.0, f(p1[i])])

            if f(p0[i]) > 0:
                lines[5].set_y(f(p0[i]) + 0.2)
            else:
                lines[5].set_y(f(p0[i]) - 0.5)

            if f(p1[i]) > 0:
                lines[6].set_y(f(p1[i]) + 0.2)
            else:
                lines[6].set_y(f(p1[i]) - 0.5)

            lines[5].set_x(p0[i] - 0.05)
            lines[5].set_text(p0_label[i])
            lines[6].set_x(p1[i] - 0.05)
            lines[6].set_text(p1_label[i])
            lines[7].set_text("")

        else:
            lines[1].set_data([pnew[i], pnew[i]], [0.0, f(pnew[i])])

            if f(pnew[i]) > 0:
                lines[7].set_y(f(pnew[i]) + 0.2)
            else:
                lines[7].set_y(f(pnew[i]) - 0.5)
            lines[7].set_x(pnew[i] - 0.05)
            lines[7].set_text(pnew_label[i])

        if p0[i] is not None:
            lines[4].set_data(
                x, (f(p1[i]) - f(p0[i])) / (p1[i] - p0[i]) * (x - p1[i]) + f(p1[i])
            )

        return

    plt.close("all")
    return FuncAnimation(fig, animate, frames=len(pnew), interval=1600)


In [60]:
secant_zero_animate(0.1, 1.4)

Порядок сходимости данного метода равен "золотому сечению"
$$
\alpha = \frac{1+\sqrt{5}}{2} \approx 1.618
$$
т.е.
$$
\lim_{k\rightarrow \infty} |\epsilon_{k+1}| \approx С |\epsilon_k|^\alpha
$$